# Cafe Sales - Data cleaning & Preparation for analysis

In this notebook, data on cafe sales was cleaned and prepared.
The main task was to prepare the data for analysis by identifying relationships between different variables.

## Load and Inspect the dataset
The data was loaded from a CSV file and an initial check was performed. At this stage, all columns are interpreted as "object", which indicates the presence of numeric values and dates with incorrect or mixed values.

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:\\Users\\lb_20\\Desktop\\kaggle\\dirty_cafe_sales.csv")

In [4]:
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [5]:
display(df.dtypes)

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [6]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


## Data type Conversion
The following conversions are applied:
- "Quantity", "Price Per Unit", "Total Spent" to numeric type
- "Transaction Date" to date and time

Invalid values are converted to "NaN" using errors="coerce".

In [8]:
df_clean = df.copy()

In [9]:
df_clean["Quantity"] = pd.to_numeric(df_clean["Quantity"], errors="coerce")
df_clean["Price Per Unit"] = pd.to_numeric(df_clean["Price Per Unit"], errors="coerce")
df_clean["Total Spent"] = pd.to_numeric(df_clean["Total Spent"], errors="coerce")
df_clean["Transaction Date"] = pd.to_datetime(df_clean["Transaction Date"], errors="coerce")

In [10]:
display(df_clean.dtypes)

Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

In [11]:
df_clean.isnull().sum()

Transaction ID         0
Item                 333
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64

## Standardization of missing values
String values such as "ERROR" and "UNKNOWN" are replaced with "NaN" to provide consistent handling of missing data across all columns.

In [13]:
df_clean = df_clean.replace(["ERROR", "UNKNOWN"], pd.NA)
df_clean

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,<NA>,<NA>,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,NaN,<NA>,2023-08-30
9996,TXN_9659401,NaN,3.0,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3.0,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [14]:
(df_clean == "ERROR").sum()
(df_clean == "UNKNOWN").sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

## Analyze missing values
The number of missing values is calculated for each column.
The overall share of missing data is also calculated.

In [16]:
missing_values_count = df_clean.isnull().sum()
missing_values_count

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [17]:
df_clean.nunique(dropna=True)

Transaction ID      10000
Item                    8
Quantity                5
Price Per Unit          6
Total Spent            17
Payment Method          3
Location                2
Transaction Date      365
dtype: int64

In [18]:
total_cells = np.product(df_clean.shape)
total_missings = missing_values_count.sum()

percent_missing = (total_missings / total_cells) * 100

print(f"The percent of missing data is: {percent_missing:.2f}%")

The percent of missing data is: 12.60%


## Restoring Values using Direct Mappings
The table of fixed menu prices was reproduced based on the information given in the task.
The idea is to fill in the missing "Item" values from the known prices and the missing "Price Per Unit" values from the known items.

In [20]:
menu_items_dict = {'Item': ['Coffee', 'Tea', 'Sandwich', 'Salad', 'Cake', 'Cookie', 'Smoothie', 'Juice'], 
                   'Price($)': [2, 1.5, 4, 5, 3, 1, 4, 3]}
menu_items_df = pd.DataFrame(menu_items_dict)
menu_items_df 

,Item,Price($)
0,Coffee,2.0
1,Tea,1.5
2,Sandwich,4.0
3,Salad,5.0
4,Cake,3.0
5,Cookie,1.0
6,Smoothie,4.0
7,Juice,3.0


In [21]:
price_to_item = {
    2.0: "Coffee",
    1.5: "Tea",
    5.0: "Salad",
    1.0: "Cookie"}

item_to_price = {
    "Coffee": 2.0, 
    "Tea": 1.5,
    "Salad": 5.0,
    "Cookie": 1.0,
    "Sandwich": 4.0,
    "Cake": 3.0,
    "Smoothie": 4.0,
    "Juice": 3.0}

In [22]:
def map_column_by_pair_value(df, dict_pair_val, in_column, out_column):
    cond = df_clean[in_column].isna() & df_clean[out_column].notna()
    df_clean.loc[cond, in_column] = df_clean.loc[cond, out_column].map(dict_pair_val)

map_column_by_pair_value(df_clean, price_to_item, "Item", "Price Per Unit")
map_column_by_pair_value(df_clean, item_to_price, "Price Per Unit", "Item")
    

In [23]:
df_clean[["Item", "Price Per Unit"]].isnull().sum()

Item              501
Price Per Unit     54
dtype: int64

## Restore Values using Column Relationships
Logical relationships between columns are applied:
- "Total Spent = Quantity × Price Per Unit"
- Missing values are calculated only when two related values are known

No values are calculated if the required information is incomplete.

In [25]:
def map_column_by_relation(df, in_column1, in_column2, out_column, l_operation):
    cond = df_clean[out_column].isna() & df_clean[in_column1].notna() & df_clean[in_column2].notna()
    df_clean.loc[cond, out_column] = l_operation(df_clean.loc[cond, in_column1], df_clean.loc[cond, in_column2])

map_column_by_relation(df_clean, "Price Per Unit", "Quantity", "Total Spent", lambda o1, o2: o1*o2)
map_column_by_relation(df_clean, "Total Spent", "Quantity", "Price Per Unit", lambda o1, o2: o1/o2)
map_column_by_relation(df_clean, "Total Spent", "Price Per Unit", "Quantity", lambda o1, o2: o1/o2)


In [26]:
df_clean[["Total Spent", "Price Per Unit", "Quantity"]].isnull().sum()

Total Spent       23
Price Per Unit     6
Quantity          23
dtype: int64

In [27]:
map_column_by_pair_value(df_clean, price_to_item, "Item", "Price Per Unit")
map_column_by_pair_value(df_clean, item_to_price, "Price Per Unit", "Item")

In [28]:
df_clean[["Item", "Price Per Unit"]].isnull().sum()

Item              480
Price Per Unit      6
dtype: int64

In [29]:
df_clean.isnull().sum()

Transaction ID         0
Item                 480
Quantity              23
Price Per Unit         6
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [30]:
df_clean[(df_clean["Quantity"]).isnull()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,NaN,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,NaN,NaN,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25
3779,TXN_7376255,<NA>,NaN,NaN,25.0,NaN,In-store,2023-05-27
4257,TXN_6470865,Coffee,NaN,2.0,NaN,Digital Wallet,Takeaway,2023-09-18


In [31]:
filter = df_clean[["Item", "Quantity", "Price Per Unit"]].isnull().all(axis=1)
indexes_filter = filter[filter].index.tolist()
df_clean[filter]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
3779,TXN_7376255,<NA>,NaN,NaN,25.0,NaN,In-store,2023-05-27
7597,TXN_1082717,<NA>,NaN,NaN,9.0,Digital Wallet,In-store,2023-12-13
9819,TXN_1208561,NaN,NaN,NaN,20.0,Credit Card,NaN,2023-08-19


## Analysis of Statistical Data by "Item" and "Quantity"
For each "Item", the following are determined:
- average value
- median value

These indicators are then used for further calculations.

In [33]:
item_to_quantity_mean = {key : value for key,value in df_clean.groupby("Item")["Quantity"].mean().items()}
item_to_quantity_mean

{'Cake': 3.047451669595782,
 'Coffee': 3.035769828926905,
 'Cookie': 2.9710982658959537,
 'Juice': 3.0008561643835616,
 'Salad': 3.0070866141732284,
 'Sandwich': 3.0425909494232477,
 'Smoothie': 3.049360146252285,
 'Tea': 3.0265339966832503}

In [34]:
item_to_quantity_median = {key : value for key,value in df_clean.groupby("Item")["Quantity"].median().items()}
item_to_quantity_median

{'Cake': 3.0,
 'Coffee': 3.0,
 'Cookie': 3.0,
 'Juice': 3.0,
 'Salad': 3.0,
 'Sandwich': 3.0,
 'Smoothie': 3.0,
 'Tea': 3.0}

In [35]:
filter = df_clean["Item"].isnull() & df_clean["Total Spent"].notnull()
indexes_filter.extend(filter[filter].index.tolist())
df_clean[filter]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6,TXN_4433211,NaN,3.0,3.0,9.0,<NA>,Takeaway,2023-10-06
8,TXN_4717867,NaN,5.0,3.0,15.0,NaN,Takeaway,2023-07-28
36,TXN_6855453,NaN,4.0,3.0,12.0,NaN,In-store,2023-07-17
61,TXN_8051289,NaN,1.0,3.0,3.0,NaN,In-store,2023-10-09
69,TXN_8471743,NaN,5.0,3.0,15.0,Digital Wallet,In-store,2023-04-06
...,...,...,...,...,...,...,...,...
9910,TXN_2338617,NaN,2.0,3.0,6.0,Digital Wallet,<NA>,2023-01-12
9918,TXN_2292088,NaN,1.0,4.0,4.0,Digital Wallet,Takeaway,2023-03-04
9946,TXN_8807600,NaN,1.0,4.0,4.0,Cash,Takeaway,2023-09-24
9981,TXN_4583012,NaN,5.0,4.0,20.0,Digital Wallet,NaN,2023-02-27


## Filling in missing "Item" values using minimum variance
For rows with known totals but missing items:
- Select the element whose expected quantity is closest to the typical values for that item
  
This step minimizes distortion of the overall structure of the elements.

In [37]:
def min_variance_quantity_by_total(total, item_to_price, item_to_quantity_mean):
    selected_lst = {}
    for item_key, price_value in item_to_price.items():
        if total % price_value == 0:
            quantity = total / price_value
            residual = abs(quantity - item_to_quantity_mean[item_key])
            selected_lst[item_key] = residual
    return min(selected_lst, key=selected_lst.get)

def min_variance_quantity_by_price(price, quantity, item_to_price, item_to_quantity_mean):
    selected_lst = {}
    for item_key, price_value in item_to_price.items():
        if price_value == price:
            residual = abs(quantity - item_to_quantity_mean[item_key])
            selected_lst[item_key] = residual
    return min(selected_lst, key=selected_lst.get)

def fill_item_by_min_variance(df, item_to_price, item_to_quantity_mean, item_to_quantity_median):
    mask = df_clean["Item"].isnull() & df_clean["Total Spent"].notnull()
    for idx, row in df_clean.loc[mask].iterrows():
        if pd.isna(row["Price Per Unit"]):
            selected = min_variance_quantity_by_total(row["Total Spent"], item_to_price, item_to_quantity_mean)
            df_clean.loc[idx, "Item"] = selected
            df_clean.loc[idx, "Quantity"] = item_to_quantity_median[selected]
            df_clean.loc[idx, "Price Per Unit"] = item_to_price[selected]
        else:
            df_clean.loc[idx, "Item"] = min_variance_quantity_by_price(row["Price Per Unit"], row["Quantity"], item_to_price, item_to_quantity_mean)
        

fill_item_by_min_variance(df_clean, item_to_price, item_to_quantity_mean, item_to_quantity_median)
filter = df_clean[["Item", "Quantity", "Price Per Unit"]].isnull().all(axis=1)
df_clean[filter]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [38]:
df_clean[df_clean.index.isin(indexes_filter)] 

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6,TXN_4433211,Juice,3.0,3.0,9.0,<NA>,Takeaway,2023-10-06
8,TXN_4717867,Cake,5.0,3.0,15.0,NaN,Takeaway,2023-07-28
36,TXN_6855453,Cake,4.0,3.0,12.0,NaN,In-store,2023-07-17
61,TXN_8051289,Juice,1.0,3.0,3.0,NaN,In-store,2023-10-09
69,TXN_8471743,Cake,5.0,3.0,15.0,Digital Wallet,In-store,2023-04-06
...,...,...,...,...,...,...,...,...
9910,TXN_2338617,Juice,2.0,3.0,6.0,Digital Wallet,<NA>,2023-01-12
9918,TXN_2292088,Sandwich,1.0,4.0,4.0,Digital Wallet,Takeaway,2023-03-04
9946,TXN_8807600,Sandwich,1.0,4.0,4.0,Cash,Takeaway,2023-09-24
9981,TXN_4583012,Smoothie,5.0,4.0,20.0,Digital Wallet,NaN,2023-02-27


In [39]:
df_clean.isnull().sum()

Transaction ID         0
Item                   3
Quantity              20
Price Per Unit         3
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [40]:
mask = df_clean[["Quantity", "Price Per Unit", "Total Spent"]].notnull().any(axis=1) & df_clean["Item"].isnull()
df_clean[mask]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2289,TXN_7524977,<NA>,4.0,NaN,NaN,<NA>,NaN,2023-12-09
4152,TXN_9646000,<NA>,2.0,NaN,NaN,NaN,In-store,2023-12-14


## Fill missing quantities using hierarchical medians
Missing "Quantity" values are filled in stages:
1. By "Location + Payment Method + Item"
2. By "Location + Item"
3. By "Item" only

Quantities are filled **only when the item is known**.

In [42]:
stats = (df_clean.groupby(["Location", "Payment Method", "Item"])["Quantity"]
         .agg(count="count", median="median", mean="mean", min="min", max="max").reset_index())

In [43]:
stats.sort_values(["Location", "Payment Method", "Item"])

,Location,Payment Method,Item,count,median,mean,min,max
0,In-store,Cash,Cake,96,3.0,3.145833,1.0,5.0
1,In-store,Cash,Coffee,75,3.0,3.013333,1.0,5.0
2,In-store,Cash,Cookie,77,3.0,3.181818,1.0,5.0
3,In-store,Cash,Juice,101,2.0,2.693069,1.0,5.0
4,In-store,Cash,Salad,110,3.0,2.863636,1.0,5.0
5,In-store,Cash,Sandwich,88,3.5,3.284091,1.0,5.0
6,In-store,Cash,Smoothie,75,3.0,2.986667,1.0,5.0
7,In-store,Cash,Tea,78,3.0,2.923077,1.0,5.0
8,In-store,Credit Card,Cake,67,4.0,3.313433,1.0,5.0
9,In-store,Credit Card,Coffee,78,3.0,2.923077,1.0,5.0


In [44]:
mask_missing = df_clean["Quantity"].isna()
df_clean.loc[mask_missing, ["Location", "Payment Method", "Item", "Quantity"]]

,Location,Payment Method,Item,Quantity
236,In-store,NaN,Salad,NaN
278,Takeaway,Cash,Juice,NaN
641,NaN,NaN,Juice,NaN
738,Takeaway,NaN,Sandwich,NaN
2796,NaN,Credit Card,Cake,NaN
3203,Takeaway,Digital Wallet,Smoothie,NaN
3224,NaN,NaN,Coffee,NaN
3401,In-store,Digital Wallet,Tea,NaN
4257,Takeaway,Digital Wallet,Coffee,NaN
5841,In-store,Digital Wallet,Cookie,NaN


In [45]:
med_lpi = df_clean.groupby(["Location", "Payment Method", "Item"])["Quantity"].median()
med_li = df_clean.groupby(["Location", "Item"])["Quantity"].median()
med_i = df_clean.groupby("Item")["Quantity"].median()

In [46]:
mask = df_clean["Quantity"].isna() & df_clean["Item"].notna()
df_clean.loc[mask, "Quantity"] = (df_clean.loc[mask].set_index(["Location", "Payment Method", "Item"]).index.map(med_lpi))

mask = df_clean["Quantity"].isna() & df_clean["Item"].notna()
df_clean.loc[mask, "Quantity"] = (df_clean.loc[mask].set_index(["Location", "Item"]).index.map(med_li))

mask = df_clean["Quantity"].isna() & df_clean["Item"].notna()
df_clean.loc[mask, "Quantity"] = df_clean.loc[mask, "Item"].map(med_i)

In [47]:
df_clean["Quantity"].isna().sum()

0

In [48]:
df_clean["Quantity"].describe()

count    10000.000000
mean         3.024950
std          1.418768
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Quantity, dtype: float64

In [49]:
df_clean.groupby(["Location", "Payment Method"])["Quantity"].median()

Location  Payment Method
In-store  Cash              3.0
          Credit Card       3.0
          Digital Wallet    3.0
Takeaway  Cash              3.0
          Credit Card       3.0
          Digital Wallet    3.0
Name: Quantity, dtype: float64

In [50]:
df_clean.isnull().sum()

Transaction ID         0
Item                   3
Quantity               0
Price Per Unit         3
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [51]:
map_column_by_relation(df_clean, "Price Per Unit", "Quantity", "Total Spent", lambda o1, o2: o1*o2)

In [52]:
df_clean.isnull().sum()

Transaction ID         0
Item                   3
Quantity               0
Price Per Unit         3
Total Spent            3
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

## Split the dataset by usability
The dataset is split into:
- **Fillable / analysis-ready rows**: "Item" and "Quantity" are known
- **Unfillable rows**: essential information is missing

The split is validated to ensure no rows are lost.

In [54]:
fillable_mask = df_clean["Item"].notna() & df_clean["Quantity"].notna()
df_fillable = df_clean[fillable_mask].copy()
df_unfillable = df_clean[~fillable_mask].copy()

In [55]:
len(df_fillable) + len(df_unfillable) == len(df_clean)

True

In [56]:
len(df_unfillable) / len(df_clean) * 100

0.03

In [57]:
df_unfillable.isna().sum().sort_values(ascending=False)

Item                3
Price Per Unit      3
Total Spent         3
Payment Method      2
Location            2
Transaction ID      0
Quantity            0
Transaction Date    0
dtype: int64

In [58]:
df_fillable.isna().sum().sort_values(ascending=False)

Location            3959
Payment Method      3176
Transaction Date     460
Transaction ID         0
Item                   0
Quantity               0
Price Per Unit         0
Total Spent            0
dtype: int64

## Evaluate data completeness
For columns "Location", "Payment Method", "Transaction Date", the number and share of known VS unknown values are calculated.

This helps determine which analyses are reliable.

In [60]:
cols = ["Location", "Payment Method", "Transaction Date"]
total = len(df_clean)

stats = (df_clean[cols].isna().sum().to_frame(name="unknown_count"))

stats["known_count"] = total - stats["unknown_count"]
stats["unknown_share"] = stats["unknown_count"] / total
stats["known_share"] = stats["known_count"] / total

stats

,unknown_count,known_count,unknown_share,known_share
Location,3961,6039,0.3961,0.6039
Payment Method,3178,6822,0.3178,0.6822
Transaction Date,460,9540,0.0460,0.9540


In [61]:
stats_percent = stats.copy()
stats_percent[["unknown_share", "known_share"]] = (stats_percent[["unknown_share", "known_share"]] * 100).round(1)

stats_percent

,unknown_count,known_count,unknown_share,known_share
Location,3961,6039,39.6,60.4
Payment Method,3178,6822,31.8,68.2
Transaction Date,460,9540,4.6,95.4
